In [1]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.firefox.options import Options
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.keys import Keys
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.remote.webelement import WebElement
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd
import yaml
from bs4 import BeautifulSoup as bs
import requests
from uuid import UUID

# from src.utils.data_log_config import logger


#### Get news links

In [ ]:
page_no = 0
url = f"https://www.fxstreet.com/news/latest/asset?q=&hPP=17&idx=FxsIndexPro&p={page_no}&dFR%5BCategory%5D%5B0%5D=News&dFR%5BTags%5D%5B0%5D=EURUSD"

In [ ]:
wd = webdriver.Chrome()
page = wd.get(url)

In [ ]:
news_boxes = wd.find_elements(
    By.CLASS_NAME,
    "ais-hits--item"
)

In [ ]:
len(news_boxes)

In [ ]:
# news_box = news_boxes[0]

def get_news_data(news_box: WebElement) -> dict:
    headline = news_box.find_element(
        By.CLASS_NAME,
        "fxs_headline_tiny"
    )
    headline_text = headline.text
    link = headline.find_element(
        By.TAG_NAME,
        "a"
    ).get_attribute("href")
    date_time = news_box.find_element(
        By.TAG_NAME,
        "time"
    ).get_attribute("datetime")

    return {
        "date_time": date_time,
        "headline": headline_text,
        "link": link,
    }


In [ ]:
news_data = get_news_data(news_box)

In [ ]:
news_data["date_time"][:10]

#### Get article text

In [ ]:
def get_article_text(article_link: str) -> str:
    news_article = requests.get(article_link)
    article_page = bs(news_article.content, "html.parser")
    return article_page.find('div', class_='fxs_article_body').text

In [ ]:
article_text = get_article_text(news_data["link"])

#### Page parsing logic

In [ ]:
def scrape_news_data(news_data_list: list = None, page_no: int = None) -> list:
    wd = webdriver.Chrome()

    if news_data_list is None:
        news_data_list = []
        page_no = 0

    

    try:
        if len(news_data_list) == 0:
            url = f"https://www.fxstreet.com/news/latest/asset?q=&hPP=17&idx=FxsIndexPro&p={page_no}&dFR%5BCategory%5D%5B0%5D=News&dFR%5BTags%5D%5B0%5D=EURUSD"
            _ = wd.get(url)
            WebDriverWait(wd, 10).until(
                EC.presence_of_all_elements_located((By.CLASS_NAME, "ais-hits--item"))
            )
            news_boxes = wd.find_elements(
                By.CLASS_NAME,
                "ais-hits--item"
            )
            for news_box in news_boxes:
                news_data_list.append(get_news_data(news_box))
            page_no += 1

        while True:
            url = f"https://www.fxstreet.com/news/latest/asset?q=&hPP=17&idx=FxsIndexPro&p={page_no}&dFR%5BCategory%5D%5B0%5D=News&dFR%5BTags%5D%5B0%5D=EURUSD"
            _ = wd.get(url)
            WebDriverWait(wd, 10).until(
                EC.presence_of_all_elements_located((By.CLASS_NAME, "ais-hits--item"))
            )
            news_boxes = wd.find_elements(
                By.CLASS_NAME,
                "ais-hits--item"
            )
            for news_box in news_boxes:
                news_data_list.append(get_news_data(news_box))
            page_no += 1
        
            if news_data_list[-1]["date_time"][:10] == "2013-12-31":
                break
            else:
                continue
        wd.quit()
        print(f"Successfully scraped {page_no + 1} pages and {len(news_data_list)} news articles")
        return news_data_list, page_no
    except Exception as e:
        wd.quit()
        print(f"scraping failed at page {page_no} || {len(news_data_list)} news articles scraped\n", e)
        return news_data_list, page_no
        

In [ ]:
news_data_list = [news_data]
news_data_list[-1]["date_time"][:10] == "2024-11-22"

In [ ]:
news_data_test = scrape_news_data()

In [ ]:
news_data_test[0]

### Back to forex factory

In [11]:
url = "https://www.forexfactory.com/calendar?range=jan1.2014-nov16.2024"

In [12]:
options = webdriver.ChromeOptions()
options.page_load_strategy = 'none'
wd = webdriver.Chrome(options=options)
wd.maximize_window()
page = wd.get(url)

WebDriverWait(wd, 10).until(
    EC.presence_of_all_elements_located((By.CLASS_NAME, "calendar__currency"))
)
wd.execute_script("window.stop();")


In [8]:
data_rows = wd.find_elements(
    By.TAG_NAME,
    "tbody"
)

In [9]:
sample = data_rows[0]

eur_data = sample.find_elements(
    By.CLASS_NAME,
    "calendar__currency"
)

one_data = sample.find_elements(
    By.TAG_NAME,
    "tr"
)
len(one_data)

11

In [10]:
row = one_data[6]

row.find_element(By.CLASS_NAME, "calendar__detail").click()
WebDriverWait(wd, 10).until(
    EC.presence_of_all_elements_located((By.CLASS_NAME, "flexposts__storydisplay-info"))
)
wd.execute_script("window.stop();")

In [8]:
wd.refresh()

In [7]:
wd.current_url

'https://www.forexfactory.com/calendar?range=jan3.2024-nov16.2024#detail=131742'

In [8]:
updated_data_rows = wd.find_elements(
    By.CLASS_NAME,
    "flexposts__storydisplay-info"
)

In [9]:
updated_data_rows[0].find_element(
    By.TAG_NAME,
    "a"
).get_attribute("href")

'https://www.forexfactory.com/news/1259958-german-labor-market-remains-resilient-despite-growth-struggles/hit'